In [ ]:
%pip install -Uqqq arize-phoenix-client arize-phoenix-otel anthropic requests beautifulsoup4 openinference-instrumentation-anthropic

In [ ]:
import os
from getpass import getpass
from itertools import chain
from pprint import pp
from textwrap import dedent

import pandas as pd
import requests
from anthropic import Anthropic
from anthropic.types.message_create_params import MessageCreateParamsBase
from bs4 import BeautifulSoup
from IPython.display import HTML, display
from openinference.instrumentation.anthropic import AnthropicInstrumentor

from phoenix.client import Client
from phoenix.client.types import PromptVersion
from phoenix.otel import register

In [ ]:
if not os.getenv("ANTHROPIC_API_KEY"):
    os.environ["ANTHROPIC_API_KEY"] = getpass("Anthropic API key: ")

In [ ]:
tracer_provider = register()
AnthropicInstrumentor().instrument(tracer_provider=tracer_provider)

Install and start a local instance of Phoenix if needed.

In [ ]:
%pip install -Uqqq arize-phoenix
import phoenix as px

px.launch_app()

# Text Generation

## Quick Start

Here's a simple LLM invocation.

In [ ]:
params = MessageCreateParamsBase(
    model="claude-3-5-haiku-latest",
    max_tokens=128,
    temperature=0,
    system="You're a coding poet",
    messages=[{"role": "user", "content": "Write a haiku about recursion in programming."}],
)
resp = Anthropic().messages.create(**params)
print(resp.content[0].text)

We can save the prompt in Phoenix.

In [ ]:
# prompt identifier should contain only alphanumeric characters, hyphens or underscores
prompt_identifier = "haiku-recursion"

prompt = Client().prompts.create(
    name=prompt_identifier,
    prompt_description="Haiku about recursion in programming",
    version=PromptVersion.from_anthropic(params),
)

We can fetch the prompt from Phoenix.

In [ ]:
prompt = Client().prompts.get(prompt_identifier=prompt_identifier)
resp = Anthropic().messages.create(**prompt.format())
print(resp.content[0].text)

## Building a moderation filter

Base on [this example](https://colab.research.google.com/github/anthropics/anthropic-cookbook/blob/09c00ae4694b1a754dac26202ec13c1190a54a52/misc/building_moderation_filter.ipynb#scrollTo=S22qtucNQaPs&line=7&uniqifier=1) from Anthropic cookbook

In [ ]:
content = """\
    You are a content moderation expert tasked with categorizing user-generated text based on the following guidelines:

    {{guidelines}}

    Here is the user-generated text to categorize:
    <user_text>{{user_text}}</user_text>

    Based on the guidelines above, classify this text as either ALLOW or BLOCK. Return nothing else.\
"""

params = MessageCreateParamsBase(
    model="claude-3-5-haiku-latest",
    max_tokens=10,
    messages=[{"role": "user", "content": dedent(content)}],
)

Save prompt in Phoenix

In [ ]:
# prompt identifier should contain only alphanumeric characters, hyphens or underscores
prompt_identifier = "content-moderation"

prompt = Client().prompts.create(
    name=prompt_identifier,
    prompt_description="Content moderation task",
    version=PromptVersion.from_anthropic(params),
)

Fetch prompt and apply to data.

In [ ]:
example_guidelines = """\
    BLOCK CATEGORY:
    - Promoting violence, illegal activities, or hate speech
    - Explicit sexual content
    - Harmful misinformation or conspiracy theories

    ALLOW CATEGORY:
    - Most other content is allowed, as long as it is not explicitly disallowed\
"""

user_comments = [
    "This movie was great, I really enjoyed it. The main actor really killed it!",
    "Delete this post now or you better hide. I am coming after you and your family.",
    "Stay away from the 5G cellphones!! They are using 5G to control you.",
    "Thanks for the helpful information!",
]

prompt = Client().prompts.get(prompt_identifier=prompt_identifier)

for comment in user_comments:
    variables = {"user_text": comment, "guidelines": example_guidelines}
    response = Anthropic().messages.create(**prompt.format(variables=variables))
    print(f"User comment: {comment}")
    print(f"Model response: {response.content[0].text}")
    print()

# Tool Use

## Article Summarization

Based on [this example](https://colab.research.google.com/github/anthropics/anthropic-cookbook/blob/09c00ae4694b1a754dac26202ec13c1190a54a52/tool_use/extracting_structured_json.ipynb#scrollTo=dcrGrM9G9CNO&line=2&uniqifier=1) from Anthropic cookbook

In [ ]:
tools = [
    {
        "name": "print_summary",
        "description": "Prints a summary of the article.",
        "input_schema": {
            "type": "object",
            "properties": {
                "author": {
                    "type": "string",
                    "description": "Name of the article author",
                },
                "topics": {
                    "type": "array",
                    "items": {"type": "string"},
                    "description": 'Array of topics, e.g. ["tech", "politics"]. Should be as specific as possible, and can overlap.',
                },
                "summary": {
                    "type": "string",
                    "description": "Summary of the article. One or two paragraphs max.",
                },
                "coherence": {
                    "type": "integer",
                    "description": "Coherence of the article's key points, 0-100 (inclusive)",
                },
                "persuasion": {
                    "type": "number",
                    "description": "Article's persuasion score, 0.0-1.0 (inclusive)",
                },
            },
            "required": [
                "author",
                "topics",
                "summary",
                "coherence",
                "persuasion",
                "counterpoint",
            ],
        },
    }
]

content = """\
    <article>
    {{article}}
    </article>

    Use the `print_summary` tool.\
"""

params = MessageCreateParamsBase(
    model="claude-3-5-haiku-latest",
    max_tokens=4096,
    tools=tools,
    tool_choice={"type": "tool", "name": "print_summary"},
    messages=[{"role": "user", "content": dedent(content)}],
)

Save prompt in Phoenix.

In [ ]:
# prompt identifier should contain only alphanumeric characters, hyphens or underscores
prompt_identifier = "article-summary"

prompt = Client().prompts.create(
    name=prompt_identifier,
    prompt_description="Article summary",
    version=PromptVersion.from_anthropic(params),
)

Download articles.

In [ ]:
articles = []
for item in ("third-party-testing", "alignment-faking"):
    response = requests.get(f"https://www.anthropic.com/news/{item}")
    soup = BeautifulSoup(response.text, "html.parser")
    articles.append({"article": " ".join([p.text for p in soup.find_all("p")])})

Fetch prompt from Phoenix and apply to data.

In [ ]:
prompt = Client().prompts.get(prompt_identifier=prompt_identifier)


def summarize(input: dict[str, str]):
    response = Anthropic().messages.create(**prompt.format(variables=input))
    for content in response.content:
        if content.type == "tool_use":
            yield content.input


res = pd.json_normalize(chain.from_iterable(map(summarize, articles)))
display(HTML(res.to_html()))

## Text classification

Base on [this example](https://colab.research.google.com/github/anthropics/anthropic-cookbook/blob/09c00ae4694b1a754dac26202ec13c1190a54a52/tool_use/extracting_structured_json.ipynb#scrollTo=QAIe78WiDN5B&line=13&uniqifier=1) from Anthropic cookbook.

In [ ]:
tools = [
    {
        "name": "print_classification",
        "description": "Prints the classification results.",
        "input_schema": {
            "type": "object",
            "properties": {
                "categories": {
                    "type": "array",
                    "items": {
                        "type": "object",
                        "properties": {
                            "name": {
                                "type": "string",
                                "description": "The category name.",
                            },
                            "score": {
                                "type": "number",
                                "description": "The classification score for the category, ranging from 0.0 to 1.0.",
                            },
                        },
                        "required": ["name", "score"],
                    },
                }
            },
            "required": ["categories"],
        },
    }
]

content = """\
    <document>
    {{text}}
    </document>

    Use the print_classification tool. The categories can be Politics, Sports, Technology, Entertainment, Business.\
"""

# https://docs.anthropic.com/en/api/messages
params = MessageCreateParamsBase(
    model="claude-3-5-haiku-latest",
    max_tokens=4096,
    tools=tools,
    tool_choice={"type": "tool", "name": "print_classification"},
    messages=[{"role": "user", "content": dedent(content)}],
)

Save prompt in Phoenix.

In [ ]:
# prompt identifier should contain only alphanumeric characters, hyphens or underscores
prompt_identifier = "document-classification"

prompt = Client().prompts.create(
    name=prompt_identifier,
    prompt_description="Document classification",
    version=PromptVersion.from_anthropic(params),
)

Fetch prompt from Phoenix and apply to data.

In [ ]:
prompt = Client().prompts.get(prompt_identifier=prompt_identifier)

variables = {
    "text": "The new quantum computing breakthrough could revolutionize the tech industry."
}
response = Anthropic().messages.create(**prompt.format(variables=variables))
for content in response.content:
    if content.type == "tool_use":
        pp(content.input)
        break

## Working with unknown keys

Based on [this example](https://colab.research.google.com/github/anthropics/anthropic-cookbook/blob/09c00ae4694b1a754dac26202ec13c1190a54a52/tool_use/extracting_structured_json.ipynb#scrollTo=QTTcOArJATAu&line=24&uniqifier=1) from Anthropic cookbook.

In [ ]:
tools = [
    {
        "name": "print_all_characteristics",
        "description": "Prints all characteristics which are provided.",
        "input_schema": {"type": "object", "additionalProperties": True},
    }
]

content = """\
    Given a description of a character, your task is to extract all the characteristics of the character and print them using the print_all_characteristics tool.

    The print_all_characteristics tool takes an arbitrary number of inputs where the key is the characteristic name and the value is the characteristic value (age: 28 or eye_color: green).

    <description>
    {{desc}}
    </description>

    Now use the print_all_characteristics tool.\
"""

params = MessageCreateParamsBase(
    model="claude-3-5-haiku-latest",
    max_tokens=4096,
    tools=tools,
    tool_choice={"type": "tool", "name": "print_all_characteristics"},
    messages=[{"role": "user", "content": dedent(content)}],
)

Save prompt in Phoenix.

In [ ]:
# prompt identifier should contain only alphanumeric characters, hyphens or underscores
prompt_identifier = "character-characteristics"

prompt = Client().prompts.create(
    name=prompt_identifier,
    prompt_description="Character characteristics",
    version=PromptVersion.from_anthropic(params),
)

Fetch prompt from Phoenix and apply to data.

In [ ]:
prompt = Client().prompts.get(prompt_identifier=prompt_identifier)

variables = {
    "desc": "The man is tall, with a beard and a scar on his left cheek. He has a deep voice and wears a black leather jacket."
}
response = Anthropic().messages.create(**prompt.format(variables=variables))
for content in response.content:
    if content.type == "tool_use":
        pp(content.input)
        break